In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import librosa
import shutil
import keras


import keras.backend as K
import tensorflow as tf
from scipy import signal

from keras.optimizers import *

from keras.regularizers import *

from keras import regularizers, optimizers
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.utils import *
from keras.callbacks import *
from sklearn.model_selection import *
# from sklearn.cross_validation import StratifiedKFold
os.environ['CUDA_VISIBLE_DEVICES']='0'
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)
from keras import backend as K
K.get_session().list_devices()

/root/anaconda3/envs/audio/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/root/anaconda3/envs/audio/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 10968078746)]

In [2]:
os.listdir('../audio-data/')
train_path = '../audio-data/audio_train/'
test_path = '../audio-data/audio_test/'
train = pd.read_csv('../audio-data/train.csv')
test = pd.read_csv('../audio-data/sample_submission.csv')
# print('training samples: ', len(os.listdir(train_path)))
# print('test samples: ', len(os.listdir(test_path)))
# print('training labels: ', len(train.label.unique()))
# print(train.head())
LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index('fname', inplace=True)
test.set_index('fname', inplace=True)
train['label_idx'] = train.label.apply(lambda x: label_idx[x])



In [3]:
class Config(object):
    def __init__(self,
                sampling_rate=44100, audio_duration=2, n_classes=41,
                use_mfcc=False, n_folds=10, learning_rate=0.0001,
                max_epochs=50, n_mfcc=20, use_log_sp=False, use_mixup=False, alpha=0.2, use_log_mel_sp=False):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_log_sp = use_log_sp
        self.use_mfcc = use_mfcc
        self.use_mixup = use_mixup
        self.use_log_mel_sp = use_log_mel_sp
        self.alpha = alpha
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            # np.floor 计算比每一个元素小或相等的最大的整数
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        elif self.use_log_sp:
            self.dim = (self.audio_duration*100-1, self.sampling_rate//100+1, 3)
        elif self.use_log_mel_sp:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        else:
            self.dim = (self.audio_length, 1)
            





In [4]:
# spectrogram.shape[0] = time*100 - 1
# spectrogram.shape[1] = rate/100 + 1
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data - min_data) / (max_data - min_data + 1e-6)
    return data - 0.5

In [ ]:
%%time
X_train = np.load('X_1d_24000x5_train.npy')
X_test = np.load('X_1d_24000x5_test.npy')
# X_train = np.load('X_1d_train.npy')
# X_test = np.load('X_1d_test.npy')
y_train = np.load('y_train.npy')

print(X_train.shape, X_test.shape, y_train.shape)

In [6]:
%%time
def normalization(x):
    mean = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    x = (x - mean) / std
    return x
X_train = normalization(X_train)
X_test = normalization(X_test)
print(X_train.shape, X_test.shape)

(9473, 120000, 1) (9400, 120000, 1)
CPU times: user 22.3 s, sys: 25.5 s, total: 47.8 s
Wall time: 46.7 s


In [7]:
%%time
def mixup(data, targets, alpha):
        size = data.shape[0]
        weight = np.random.beta(alpha, alpha, size)
      
#         x_weight = weight.reshape(size, 1, 1, 1)
        x_weight = weight.reshape(size, 1, 1)
        y_weight = weight.reshape(size, 1)
        index = np.random.permutation(size)
        x1, x2 = data, data[index]
        x = x1 * x_weight + x2 * (1 - x_weight)
        y1, y2 = targets, targets[index]
        y = y1 * y_weight + y2 * (1 - y_weight)
        return x, y
X_train_mixup, y_train_mixup = mixup(X_train, y_train, 2)
print(X_train_mixup.shape, y_train_mixup.shape)

(9473, 120000, 1) (9473, 41)
CPU times: user 7.13 s, sys: 15.7 s, total: 22.8 s
Wall time: 22.3 s


In [8]:


def BLOCK(seq, filters): # 定义网络的Block
    cnn = Conv1D(filters*2, 3, padding='SAME', dilation_rate=1, activation='relu')(seq)
    cnn = Lambda(lambda x: x[:,:,:filters] + x[:,:,filters:])(cnn)
    cnn = Conv1D(filters*2, 3, padding='SAME', dilation_rate=2, activation='relu')(cnn)
    cnn = Lambda(lambda x: x[:,:,:filters] + x[:,:,filters:])(cnn)
    cnn = Conv1D(filters*2, 3, padding='SAME', dilation_rate=4, activation='relu')(cnn)
    cnn = Lambda(lambda x: x[:,:,:filters] + x[:,:,filters:])(cnn)
    if int(seq.shape[-1]) != filters:
        seq = Conv1D(filters, 1, padding='SAME')(seq)
    seq = add([seq, cnn])
    return seq


def get_model():
#     with tf.device('/cpu:0'):
        #搭建模型，就是常规的CNN加残差加池化
#         input_tensor = Input(shape=(16000, 1))
    x_in = Input(shape=(24000*5, 1))

    seq = BLOCK(x_in, 32)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 32)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 64)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 64)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 128)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 128)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 256)
    seq = MaxPooling1D(2)(seq)
    seq = BatchNormalization()(seq)
    seq = BLOCK(seq, 256)
#     seq = Dropout(0.5, (128, int(seq.shape[1]), 1))(seq)
#     seq = Dropout(0.5)(seq)
    seq = GlobalAveragePooling1D()(seq)
    seq = BatchNormalization()(seq)
#     seq = Dense(256, activation='relu')(seq)
#     seq = Dropout(0.5)(seq)
#     seq = Dense(256, activation='relu')(seq)
#     seq = Dropout(0.5)(seq)
    
    output = Dense(41, activation='softmax')(seq)

    model = Model(inputs=x_in, outputs=output)
    print(len(model.layers))
    model.summary()
    return model

In [9]:
class ParallelModelCheckpoint(ModelCheckpoint):
    def __init__(self,model,filepath, monitor='val_loss', verbose=1,
                 save_best_only=True, save_weights_only=True,
                 mode='auto', period=1):
        self.single_model = model
        super(ParallelModelCheckpoint,self).__init__(filepath, monitor, verbose,save_best_only, save_weights_only,mode, period)

    def set_model(self, model):
        super(ParallelModelCheckpoint,self).set_model(self.single_model)


class CustomModelCheckpoint(keras.callbacks.Callback):

    def __init__(self, model, path):
        self.model = model
        self.path = path
        self.best_loss = np.inf

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs['val_loss']
        if val_loss < self.best_loss:
            print("\nValidation loss decreased from {} to {}, saving model".format(self.best_loss, val_loss))
            self.model.save_weights(self.path, overwrite=True)
            self.best_loss = val_loss


In [10]:
%%time

            
config = Config(sampling_rate=24000, audio_duration=5, n_classes=41, n_folds=10, max_epochs=300, n_mfcc=128)
PREDICTION_FOLDER = "predictions_1d_24000x5_mixup_fully_trained"
CHECKPOINT_FOLDER = 'checkpionts_1d_24000x5_mixup_fully_trained'

if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 44

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = np.r_[X_train[train_split], X_train_mixup[train_split]]
#     y = np.r_[y_train[train_split], y_train_mixup[train_split]]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = X_train[train_split]
#     y = y_train[train_split]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
i = 99
X = np.r_[X_train, X_train_mixup]
y = np.r_[y_train, y_train_mixup]
print('train shape: ', X.shape, y.shape)
# print('val shape: ', X_val.shape, y_val.shape)

print("Fold: ", i)

model = get_model()
parallel_model = multi_gpu_model(model, gpus=3)

#     model.load_weights('checkpoint_1d_24000x5' + '/best_99.h5')

adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6, amsgrad=True)
parallel_model.compile(loss='categorical_crossentropy',
                   optimizer=adam,
                     metrics=['acc'])


#     checkpoint = ModelCheckpoint(CHECKPOINT_FOLDER+'/best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
#     checkpoint = CustomModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
checkpoint = ParallelModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i, monitor='loss', 
                                     verbose=1, save_best_only=True, save_weights_only=True)
early = EarlyStopping(monitor="loss", mode="min", patience=15)
tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
rlrop = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5)
log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
callbacks_list = [checkpoint, early, tb, rlrop, log]

history = parallel_model.fit(X, y, validation_data=None, callbacks=callbacks_list, 
                    batch_size=batch_size, epochs=config.max_epochs, shuffle=True)


# model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
# #     parallel_model = multi_gpu_model(model, gpus=2)



# # Save train predictions
# predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
# np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

# # Save test predictions
# predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
# np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

# # Make a submission file
# top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
# predicted_labels = [' '.join(list(x)) for x in top_3]
# test['label'] = predicted_labels
# test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)






train shape:  (18946, 120000, 1) (18946, 41)
Fold:  99
78
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120000, 1)    0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 120000, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 120000, 32)   0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 120000, 64)   6208        lambda_1[0][0]                   
___________________________________________________

Epoch 1/300
18946/18946 [==============================] - 345s 18ms/step - loss: 3.6708 - acc: 0.0940

Epoch 00001: loss improved from inf to 3.67075, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 2/300
18946/18946 [==============================] - 337s 18ms/step - loss: 3.1980 - acc: 0.1689

Epoch 00002: loss improved from 3.67075 to 3.19799, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 3/300
18946/18946 [==============================] - 335s 18ms/step - loss: 3.0288 - acc: 0.2248

Epoch 00003: loss improved from 3.19799 to 3.02879, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 4/300
18946/18946 [==============================] - 336s 18ms/step - loss: 2.9249 - acc: 0.2576

Epoch 00004: loss improved from 3.02879 to 2.92488, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 5/300
18946/18946 [==============================] - 334s 18ms/step - loss: 2.7960 - acc: 0.29

18946/18946 [==============================] - 338s 18ms/step - loss: 1.7449 - acc: 0.5986

Epoch 00038: loss improved from 1.76196 to 1.74493, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 39/300
18946/18946 [==============================] - 338s 18ms/step - loss: 1.7366 - acc: 0.5998

Epoch 00039: loss improved from 1.74493 to 1.73659, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 40/300
18946/18946 [==============================] - 337s 18ms/step - loss: 1.7447 - acc: 0.5970

Epoch 00040: loss did not improve
Epoch 41/300
18946/18946 [==============================] - 337s 18ms/step - loss: 1.7041 - acc: 0.6065

Epoch 00041: loss improved from 1.73659 to 1.70415, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 42/300
18946/18946 [==============================] - 338s 18ms/step - loss: 1.6656 - acc: 0.6187

Epoch 00042: loss improved from 1.70415 to 1.66565, saving model to checkpionts_1d_24000

18946/18946 [==============================] - 338s 18ms/step - loss: 0.7613 - acc: 0.8470

Epoch 00114: loss improved from 0.76330 to 0.76127, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 115/300
18946/18946 [==============================] - 337s 18ms/step - loss: 0.7547 - acc: 0.8491

Epoch 00115: loss improved from 0.76127 to 0.75465, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 116/300
18946/18946 [==============================] - 338s 18ms/step - loss: 0.7551 - acc: 0.8487

Epoch 00116: loss did not improve
Epoch 117/300
18946/18946 [==============================] - 338s 18ms/step - loss: 0.7478 - acc: 0.8492

Epoch 00117: loss improved from 0.75465 to 0.74781, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 118/300
18946/18946 [==============================] - 338s 18ms/step - loss: 0.7397 - acc: 0.8522

Epoch 00118: loss improved from 0.74781 to 0.73971, saving model to checkpionts_1d_2

18946/18946 [==============================] - 331s 17ms/step - loss: 0.5610 - acc: 0.8743

Epoch 00201: loss improved from 0.56213 to 0.56103, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 202/300
18946/18946 [==============================] - 331s 17ms/step - loss: 0.5659 - acc: 0.8734

Epoch 00202: loss did not improve
Epoch 203/300
18946/18946 [==============================] - 331s 17ms/step - loss: 0.5611 - acc: 0.8734

Epoch 00203: loss did not improve
Epoch 204/300
18946/18946 [==============================] - 332s 18ms/step - loss: 0.5555 - acc: 0.8747

Epoch 00204: loss improved from 0.56103 to 0.55547, saving model to checkpionts_1d_24000x5_mixup_fully_trained/best_99.h5
Epoch 205/300
18946/18946 [==============================] - 331s 17ms/step - loss: 0.5610 - acc: 0.8749

Epoch 00205: loss did not improve
Epoch 206/300
18946/18946 [==============================] - 331s 17ms/step - loss: 0.5541 - acc: 0.8746

Epoch 00206: loss improved from 

KeyboardInterrupt: 

In [12]:
model.load_weights(CHECKPOINT_FOLDER + '/best_99.h5')
parallel_model = multi_gpu_model(model, gpus=3)




# # Save train predictions
# predictionparallel_modelparallel_model.predict(X_train, batch_size=batch_size, verbose=1)
# np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

# Save test predictions
predictions = parallel_model.predict(X_test, batch_size=batch_size, verbose=1)
np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test['label'] = predicted_labels
test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)
print('done')


9400/9400 [==============================] - 69s 7ms/step
done


In [14]:

pred_list_mixup = []
pred_list = []

# for i in range(10):
#     pred_list.append(np.load("predictions_log_mel_sp_24000x5_mixup/test_predictions_%d.npy"%i))
# for i in range(7):
#     pred_list.append(np.load("../predictions_1d_24000x5/test_predictions_%d.npy"%i))
# for i in range(7):
#     pred_list.append(np.load('predictions_24000x2_log_sp/test_predictions_%d.npy'%i))

# for i in range(7):
#     pred_list.append(np.load('predictions_log_mel_sp_24000x5_image_gen/test_predictions_%d.npy'%i))
pred_list.append(np.load('predictions_1d_24000x5_mixup_fully_trained/test_predictions_99.npy'))

# prediction_mixup = np.ones_like(pred_list[0])
prediction = np.ones_like(pred_list[0])



# for pred in pred_list_mixup:
#     prediction_mixup = prediction_mixup * pred
for pred in pred_list:
    prediction = prediction * pred

    
# prediction_mixup = prediction_mixup**(1./len(pred_list_mixup))
prediction = prediction**(1./len(pred_list))


# prediction = 0.5 * prediction + prediction_mixup

# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('../audio-data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("1d_fully.csv", index=False)





In [15]:
%%time

            
config = Config(sampling_rate=24000, audio_duration=5, n_classes=41, n_folds=10, max_epochs=300, n_mfcc=128)
PREDICTION_FOLDER = "predictions_1d_24000x5_mixup"
CHECKPOINT_FOLDER = 'checkpionts_1d_24000x5_mixup'

if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)
if not os.path.exists(CHECKPOINT_FOLDER):
    os.mkdir(CHECKPOINT_FOLDER)
if os.path.exists('logs/' + PREDICTION_FOLDER):
    shutil.rmtree('logs/' + PREDICTION_FOLDER)

batch_size = 15

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=2)


for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
    X = np.r_[X_train[train_split], X_train_mixup[train_split]]
    y = np.r_[y_train[train_split], y_train_mixup[train_split]]
    X_val = X_train[val_split]
    y_val = y_train[val_split]
    
# for i, (train_split, val_split) in enumerate(skf.split(X_train, np.argmax(y_train, axis=1))):
#     X = X_train[train_split]
#     y = y_train[train_split]
#     X_val = X_train[val_split]
#     y_val = y_train[val_split]
    
    print('train shape: ', X.shape, y.shape)
    print('val shape: ', X_val.shape, y_val.shape)

    print("Fold: ", i)
    
    model = get_model()
#     parallel_model = multi_gpu_model(model, gpus=2)
    
#     model.load_weights('checkpoint_1d_24000x5' + '/best_99.h5')
    
    adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6, amsgrad=True)
    model.compile(loss='categorical_crossentropy',
                       optimizer=adam,
                         metrics=['acc'])


    #     checkpoint = ModelCheckpoint(CHECKPOINT_FOLDER+'/best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
#     checkpoint = CustomModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i)
    checkpoint = ParallelModelCheckpoint(model, CHECKPOINT_FOLDER + '/best_%d.h5'%i, monitor='val_loss', 
                                         verbose=1, save_best_only=True, save_weights_only=True)
    early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
    tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%d'%i, write_graph=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    log = CSVLogger(PREDICTION_FOLDER + '/log_%d.csv'%i)
    callbacks_list = [checkpoint, early, tb, rlrop, log]
    
    history = model.fit(X, y, validation_data=(X_val, y_val), callbacks=callbacks_list, 
                        batch_size=batch_size, epochs=config.max_epochs, shuffle=True)
    

    model.load_weights(CHECKPOINT_FOLDER + '/best_%d.h5'%i)
#     parallel_model = multi_gpu_model(model, gpus=2)


    
    # Save train predictions
    predictions = model.predict(X_train, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

    # Save test predictions
    predictions = model.predict(X_test, batch_size=batch_size, verbose=1)
    np.save(PREDICTION_FOLDER + "/test_predictions_%d.npy"%i, predictions)

    # Make a submission file
    top_3 = np.array(LABELS)[np.argsort(-predictions, axis=1)[:, :3]]
    predicted_labels = [' '.join(list(x)) for x in top_3]
    test['label'] = predicted_labels
    test[['label']].to_csv(PREDICTION_FOLDER + "/predictions_%d.csv"%i)





train shape:  (17036, 120000, 1) (17036, 41)
val shape:  (955, 120000, 1) (955, 41)
Fold:  0
78
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 120000, 1)    0                                            
__________________________________________________________________________________________________
conv1d_169 (Conv1D)             (None, 120000, 64)   256         input_7[0][0]                    
__________________________________________________________________________________________________
lambda_145 (Lambda)             (None, 120000, 32)   0           conv1d_169[0][0]                 
__________________________________________________________________________________________________
conv1d_170 (Conv1D)             (None, 120000, 64)   6208        lambda_145[0][0]                 
_____________

Train on 17036 samples, validate on 955 samples
Epoch 1/300
17036/17036 [==============================] - 797s 47ms/step - loss: 3.6714 - acc: 0.0998 - val_loss: 4.0939 - val_acc: 0.0503

Epoch 00001: val_loss improved from inf to 4.09392, saving model to checkpionts_1d_24000x5_mixup/best_0.h5
Epoch 2/300
17036/17036 [==============================] - 784s 46ms/step - loss: 3.1806 - acc: 0.1736 - val_loss: 3.6243 - val_acc: 0.1131

Epoch 00002: val_loss improved from 4.09392 to 3.62433, saving model to checkpionts_1d_24000x5_mixup/best_0.h5
Epoch 3/300
17036/17036 [==============================] - 784s 46ms/step - loss: 3.1242 - acc: 0.1966 - val_loss: 3.2847 - val_acc: 0.1173

Epoch 00003: val_loss improved from 3.62433 to 3.28470, saving model to checkpionts_1d_24000x5_mixup/best_0.h5
Epoch 4/300
17036/17036 [==============================] - 779s 46ms/step - loss: 3.0236 - acc: 0.2251 - val_loss: 3.2492 - val_acc: 0.1571

Epoch 00004: val_loss improved from 3.28470 to 3.24915, sav


Epoch 00040: val_loss did not improve
Epoch 41/300
17036/17036 [==============================] - 778s 46ms/step - loss: 1.7383 - acc: 0.6017 - val_loss: 1.5929 - val_acc: 0.6482

Epoch 00041: val_loss improved from 1.91603 to 1.59289, saving model to checkpionts_1d_24000x5_mixup/best_0.h5
Epoch 42/300
17036/17036 [==============================] - 779s 46ms/step - loss: 1.6898 - acc: 0.6167 - val_loss: 1.6042 - val_acc: 0.6723

Epoch 00042: val_loss did not improve
Epoch 43/300
17036/17036 [==============================] - 779s 46ms/step - loss: 1.6615 - acc: 0.6244 - val_loss: 1.5553 - val_acc: 0.6565

Epoch 00043: val_loss improved from 1.59289 to 1.55534, saving model to checkpionts_1d_24000x5_mixup/best_0.h5
Epoch 44/300
17036/17036 [==============================] - 779s 46ms/step - loss: 1.6438 - acc: 0.6279 - val_loss: 1.5716 - val_acc: 0.6586

Epoch 00044: val_loss did not improve
Epoch 45/300
17036/17036 [==============================] - 778s 46ms/step - loss: 1.6290 - acc

KeyboardInterrupt: 